# Serie 2

Dans cette série on va voir comment faire de la déanonymisation en utilisant deux
sources:
- imdb avec des évaluations publiques faites par les utilisateurs
- netflix avec des évaluations anonymisées et publié par Netflix, avec plus d'évaluations

Les deux sources sont faites dans deux étapes:
- `_small.csv` avec peu d'entrées
- `_big.csv` avec beaucoup d'entrées

Tous les noms sont aléatoire et les évaluations ne correspondent en aucun cas à des gens réels ;)

# Exercice 1

Dans un premier temps on va simplement charger les données et les afficher.

## 1. Connaissance

Vous trouvez deux jeux de données:
- netflix_small.csv - contient quatre colonnes: nom (anonymisé), film (anonymisé), date, évaluation
- imdb_small.csv - contient les mêmes quatre colonnes, mais non-anonymisées

En double-cliquant dessus, vous pouvez voir leur contenu.

**Questions**

1. laquelle est plus longue?
2. voyez vous l'anonymisation?

Il y a une méthode `load_csv` qui retourne une liste de `dict`, et qui est déjà utilisée pour charger les deux bases de données.

La méthode `print_list` affiche une liste, une entrée par ligne. C'est un peu plus convivial que `print(imdb)`.

Finalement il y a une commande pour `sort` la liste, ici on le fait selon le `rating`.

**Questions**

3. Avec le triage selon le `rating` vous pouvez déjà approcher les listes - mais selon quelle colonne faudrait-il trier pour bien comparer les deux bases de données?
4. Ici les bases de données ont les mêmes dates et les mêmes `rating` - comment est-ce que c'est dans une vraie base de données? Est-ce que vous avez une idée comment on pourrait procéder?

On verra dans l'exercice prochain comment faire si les deux listes contiennent des dates répétées.

In [ ]:
import create_db, csv

def load_csv(name: str) -> list:
    ret = []
    
    with open(name, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            ret.append(row)
            
    return ret

def print_list(l: list):
    for e in l:
        print(e)

imdb = load_csv("imdb_small.csv")
netflix = load_csv("netflix_small.csv")
print("Raw list:")
print_list(imdb)

imdb_sorted = sorted(imdb, key = lambda line: line['rating'])
print("\nSorted list:")
print_list(imdb_sorted)

### Réponses

1. La base de données de Netflix est plus longue, car elle contient plus d'évaluations
2. On voit que les noms des utilisateurs et les films ont été remplacés
3. En triant selon le `date` on pourrait mieux désanonymiser la base de données Netflix
4. Dans une base de données tirée de la vraie vie, les dates et les évaluations seront au moins un peu différents

## 2. Compréhension

Maintenant que vous avez pu trier les listes, essayez de faire la correspondance pour au moins un utilisateur et tous les films de cet utilisateur.

**Questions**

1. Quelle est la correspondance entre l'utilisateur et le nom anonymisé dans la base de données de Netflix?
2. Quels sont ses films évalués sur IMDB? Et quels films a-t-il·elle aussi regardé·e sur Netflix?

In [ ]:
imdb_sorted = sorted(imdb, key = lambda line: line['date'])
print("\nSorted imdb:")
print_list(imdb_sorted)

netflix_sorted = sorted(netflix, key = lambda line: line['date'])
print("\nSorted netflix:")
print_list(netflix_sorted)

### Réponses

En triant les deux bases de données par la date, on peut vérifier manuellement.
Voici les correspondances directes entre les deux bases de données:

b0184e37 -> Andrea Terra
* 622aa154 -> The Lord of the Rings: The Return of the King (2003)
* afee44e6 -> Star Wars: Episode V - The Empire Strikes Back (1980)
* af8ceb8b -> Fight Club (1999)
* 8c62add9 -> Forrest Gump (1994)
* 72005112 -> The Dark Knight (2008)
* 71e881b0 -> 

9c000427 -> Andrea Rordorf
* 45ef8414 -> 
* e45cc73b -> 12 Angry Men (1957)
* dfe75058 -> Pulp Fiction (1994)
* 894b36c1 -> The Shawshank Redemption (1994)
* 71e881b0 -> The Matrix (1999)
* ebb77707 -> Inception (2010)

Pour les films manquants, il faut trouver une autre correspondance. Même si c'est d'un autre utilisateur, ça doit marcher.
* 45ef8414 -> Ce film ne se trouve dans aucune autre correspondance de dates, on ne peut donc pas savoir qu'est-ce que c'est comme film...
* 71e881b0 -> The Matrix (1999)

## 3. Application

La prochaine étape est bien plus proche de la réalité. Les bases de données sont maintenant fait avec
1024 utilisateurs, et chaque utilisateur a évalué 10 films sur IMDB et 12 sur Netflix.
Le but est donc de trouver les 2 films qu'on ne connaît pas.
Avec un total de 1024 évaluations de 10 films sur une année, on ne peut plus se fier à la date, parce qu'on trouvera trop de doublons.

### Faisable - fréquence des évaluations

Mais vous pouvez encore faire une attaque en utilisant la fréquence des films:
on suppose que la fréquence des évaluations est la même dans IMDB et Netflix. Si on arrive à trier les films selon leur nombre d'évaluations, on peut faire un matching de la liste.

**Questions**

1. Donnez les correspondances des 20 films entre le nom complet dans IMDB et le hash de Netflix
2. Quelles sont les différences avec la réalité par rapport aux évaluations?
3. Comment faudrait-il procéder pour quand même trouver les films en réalité?

### Très avancé - trouver les noms

Pour trouver les noms des personnes, on peut faire deux choses:

#### Solution 1

Prendre un nom d'utilisateur·trice dans IMDB, et faire une correspondance avec les films qu'il·elle a évalué avec les films évalués par un nom anonymisé dans Netflix

**Questions**

4. Quelles sont les simplifications faites?

#### Solution 2

Faire une correspondance en ne prenant pas seulement les noms des films, mais aussi les dates et les évaluations. Cette étape requiert des statistiques assez avancées - même les étudiants en Master à l'EPFL avaient de la peine ;) Si vous arrivez, faites-le savoir!

### Hint

Dans le fichier `create_db.py` j'ai mis des méthodes pour m'assurer que l'évaluation par des fréquences est possible. Parce que c'est en fait pas tout à fait simple de créer un exercice qui marche ;)

Donc vous pouvez vous laissez inspirer par les méthodes `sort_list` pour la fréquence et `uniq_choice` pour trouver les noms.

In [ ]:
from collections import Counter

imdb = load_csv("imdb_big.csv")
netflix = load_csv("netflix_big.csv")

# Choisit la colonne 'movie', compte l'occurrence de chaque élément unique,
# et trie la liste selon l'occurrence des éléments, commençant avec le plus
# frèquent.
def sort_frequency(db):
    movies = list(map(lambda m: m['movie'], db))
    count = Counter(movies)
    return sorted(count.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)

# Créer un dictionnaire pour traduire depuis un film dans le db1 à un
# film dans db2.
def create_film_translation(db1, db2):
    # Créer les deux listes triées selon la fréquence
    movies_db1 = sort_frequency(db1)
    movies_db2 = sort_frequency(db2)

    # Créer un dictionnaire de traduction en inter
    zipped = zip(movies_db1, movies_db2)
    return dict(map(lambda x: [x[0][0], x[1][0]], zipped))

# Cherche un identifiant d'un nom dans db2 à partir d'un nom dans la
# db1. La recherche est faite en passant à travers tous les films que
# l'utilisateur à évalué dans la db1, et en faisant l'intersection des
# utilisateurs dans db2 qui ont évalué ce même film.
def search_name_id(db1, db2, name):
    translate = create_film_translation(db1, db2)
    # Trouver tous les films du premier "name" dans la base IMDB
    movies_name = list(map(lambda l: l['movie'], filter(lambda l: l['name'] == name, imdb)))
    movies_hash = list(map(lambda m: translate[m], movies_name))
    # Pour chaque film, liste les utilisateurs qui ont évalué ce film
    movie_to_users = \
      dict(map(lambda h: [h, 
        list(map(lambda l: l['name'], 
           filter(lambda n: n['movie'] == h, netflix)))], 
        movies_hash))
    names = set(map(lambda l: l['name'], netflix))

    # Faire l'intersection des utilisateurs qui ont vu tous les films en question
    for movie in movies_hash:
        names = names.intersection(movie_to_users[movie])

    if len(names) != 1:
        raise(BaseException("There should only be one name left, but got", len(names)))
        
    return list(names)[0]

# Cherche les films dans les deux bsae de données en traduisant
# les films dans la représentation de la base de données db1.
# Puis retourne la différence entre les deux listes de films.
def search_missing(db1, db2, name_db1, name_db2):
    translate = create_film_translation(db2, db1)
    movies_db1 = \
      set(map(lambda f: f['movie'], 
        filter(lambda l: l['name'] == name_db1, db1)))
    movies_db2 = \
      set(map(lambda f: translate[f['movie']], 
        filter(lambda l: l['name'] == name_db2, db2)))
        
    if len(movies_db1) > len(movies_db2):
        raise(BaseException("More movies in first db"))
        
    return movies_db2.difference(movies_db1)

In [ ]:
# Faire la recherche d'un nom et des films "secrètes".
# TODO: trouver pourquoi ça ne marche pas avec tous les noms...
name_imdb = imdb[0]['name']
name_netflix = search_name_id(imdb, netflix, name_imdb)
missing = search_missing(imdb, netflix, name_imdb, name_netflix)

print("User", name_imdb, "on IMDB corresponds to user", name_netflix, "on Netflix")
print("Secret films of user", name_imdb, "are", missing)

## Réponses

1. `print( create_film_translation(imdb, netflix)`
2. Dans cette exercice, les évaluations ont la même valeur et la même date dans les deux bases de données, ce qui n'est pas le cas dans les base de données réelles.
3. En réalité il faudrait procéder à utiliser des corrélations entre les éavaluations et travailler avec des probabilités, au lieu de trouver des correspondances exactes.
4. On suppose que chaque utilisateur évalue 10 films dans l'IMDB et les même films + deux dans Netflix. En plus on suppose que la liste de fréquence d'évaluation est la même dans IMDB et dans Netflix.